In [11]:
import pandas as pd

df_adm = pd.read_csv('../mimic-iv/hosp/admissions.csv')
df_icu = pd.read_csv('../mimic-iv/icu/icustays.csv')
df_cost = pd.read_csv('../resources/final_drug_mapping.csv')
df_presc = pd.read_csv('../mimic-iv/hosp/prescriptions.csv')

df_cost['drug'] = df_cost['drug'].str.lower().str.strip()
df_presc['drug'] = df_presc['drug'].str.lower().str.strip()
df_cost_joined = pd.merge(df_presc, df_cost, on='drug', how='left')

df_adm['hadm_id'] = df_adm['hadm_id'].astype(str)
df_icu['hadm_id'] = df_icu['hadm_id'].astype(str)
df_cost_joined['hadm_id'] = df_cost_joined['hadm_id'].astype(str)

medicine_costs = df_cost_joined.groupby('hadm_id')['cost'].sum().reset_index().rename(columns={'cost': 'medicine_cost'})

fact_icustay = pd.merge(df_adm, df_icu, on='hadm_id', how='inner')

fact_icustay = fact_icustay[['hadm_id', 'hospital_expire_flag', 'los', 'intime', 'outtime']]
fact_icustay = fact_icustay.rename(columns={'los': 'total_icu_days'})

fact_icustay_enhanced = pd.merge(fact_icustay, medicine_costs, on='hadm_id', how='left')

fact_icustay_enhanced['medicine_cost'] = fact_icustay_enhanced['medicine_cost'].fillna(0)

fact_icustay_enhanced['total_cost'] = fact_icustay_enhanced['medicine_cost']

fact_icustay_enhanced['icu_admit_date'] = pd.to_datetime(fact_icustay_enhanced['intime'])
fact_icustay_enhanced['icu_discharge_date'] = pd.to_datetime(fact_icustay_enhanced['outtime'])

print("Enhanced Fact ICU Stay DataFrame:")
fact_icustay_enhanced

Enhanced Fact ICU Stay DataFrame:


,hadm_id,hospital_expire_flag,total_icu_days,intime,outtime,medicine_cost,total_cost,icu_admit_date,icu_discharge_date
0,24181354,0,4.952106,2196-02-24 17:07:00,2196-02-29 15:58:02,38386.491395,38386.491395,2196-02-24 17:07:00,2196-02-29 15:58:02
1,25926192,0,2.280752,2153-09-19 09:54:49,2153-09-21 16:39:06,29480.314029,29480.314029,2153-09-19 09:54:49,2153-09-21 16:39:06
2,22942076,1,1.751366,2111-11-14 00:19:12,2111-11-15 18:21:10,11895.460325,11895.460325,2111-11-14 00:19:12,2111-11-15 18:21:10
3,22942076,1,0.023727,2111-11-13 23:40:00,2111-11-14 00:14:10,11895.460325,11895.460325,2111-11-13 23:40:00,2111-11-14 00:14:10
4,21606243,0,1.206481,2113-08-04 18:47:42,2113-08-05 23:45:02,7439.054191,7439.054191,2113-08-04 18:47:42,2113-08-05 23:45:02
...,...,...,...,...,...,...,...,...,...
135,24294180,0,1.171609,2134-09-07 18:03:58,2134-09-08 22:11:05,9168.786753,9168.786753,2134-09-07 18:03:58,2134-09-08 22:11:05
136,24745425,0,1.167824,2187-07-29 09:41:11,2187-07-30 13:42:51,14528.290531,14528.290531,2187-07-29 09:41:11,2187-07-30 13:42:51
137,22168393,0,1.148981,2145-09-28 20:59:43,2145-09-30 00:34:15,13129.057691,13129.057691,2145-09-28 20:59:43,2145-09-30 00:34:15
138,27708593,0,0.706748,2177-03-15 05:49:26,2177-03-15 22:47:09,11502.603511,11502.603511,2177-03-15 05:49:26,2177-03-15 22:47:09


In [57]:

import pandas as pd
from rapidfuzz import process, fuzz

# --- Step 1: Load the CSV files ---
# Replace file paths with your actual paths
df_labitems = pd.read_csv('../mimic-iv/hosp/d_labitems.csv')             # Contains lab test definitions
df_mapping = pd.read_csv('../resources/lab_tests_cost.csv')       # Contains lab test names (e.g., possibly cost-related)

# --- Step 2: Standardize the lab test names ---
# For df_labitems, assume the lab test name is in a column called 'label'
df_labitems['label'] = df_labitems['label'].str.lower().str.strip()

# For df_mapping, assume the lab test name is in a column called 'lab_test_name'
df_mapping['category'] = df_mapping['category'].str.lower().str.strip()

# --- Step 3: Define a fuzzy matching function ---
def fuzzy_match_lab(test_name, choices, threshold=80):
    """
    Given a lab test name and a list of choices,
    return the best matching choice and its score if it meets the threshold.
    """
    if pd.isnull(test_name):
        return None, None
    best_match = process.extractOne(test_name, choices, scorer=fuzz.token_set_ratio)
    if best_match and best_match[1] >= threshold:
        return best_match[0], best_match[1]
    return None, None

# --- Step 4: Fuzzy match each lab test from df_mapping to df_labitems ---
# Create a list of standardized lab test names from df_labitems
choices = set(df_labitems['label'].tolist())
fuzzy_results = []
for idx, row in df_mapping.iterrows():
    test_name = row['category']
    match, score = fuzzy_match_lab(test_name, choices, threshold=80)
    fuzzy_results.append({
        'category': row['category'],
        'label': match,
        'price': 0 if match is None else row['price']
    })

df_fuzzy = pd.DataFrame(fuzzy_results)
df_labitems_cost = pd.merge(df_labitems, df_fuzzy[['label', 'price']], on='label', how='left')
df_labitems_cost = df_labitems_cost.drop_duplicates(subset='label')
df_labitems_cost = df_labitems_cost[['label', 'category', 'price']]
default_prices = {
    'blood gas': 8,
    'chemistry': 75,
    'hematology': 100
}

def fill_default_price(row):
    # If the price is 0 (or NaN), try to get the default for that category.
    if pd.isnull(row['price']) or row['price'] == 0:
        return default_prices.get(row['category'].lower(), row['price'])
    return row['price']

df_labitems_cost['price'] = df_labitems_cost.apply(fill_default_price, axis=1)
output_csv = '../resources/lab_tests_mapping.csv'
df_labitems_cost.to_csv(output_csv, index=False)